In [50]:
## Connect to DB server on AWS
import mysql.connector 
import config
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = 'crime_nyc'
)
cursor = cnx.cursor()
cnx.database = 'crime_nyc'

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import itertools

In [52]:
query = ("SELECT * FROM crime2019;")
cursor.execute(query)
#cursor.close()
#cnx.close()

In [53]:
result = cursor.fetchall()

In [71]:
df = pd.DataFrame(result)
df.columns = [x[0] for x in cursor.description]
df

,index,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,...,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,zipcode,med_income
0,0,203078287,09/30/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1600500,F,M,...,0,25-44,M,BLACK HISPANIC,990563,203120,40.724200,-73.977226,10009,59929.0
1,1,203072424,09/30/2019,268.0,CRIMINAL MIS 2 & 3,121.0,CRIMINAL MISCHIEF & RELATED OF,PL 1450502,F,Q,...,0,18-24,M,BLACK,1040611,190715,40.689974,-73.796769,11433,42887.0
2,2,203061215,09/30/2019,905.0,"INTOXICATED DRIVING,ALCOHOL",347.0,INTOXICATED & IMPAIRED DRIVING,VTL11920U3,M,S,...,0,18-24,M,WHITE,962989,160112,40.606129,-74.076570,10305,70758.0
3,3,203061218,09/30/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1601502,F,S,...,0,25-44,M,WHITE HISPANIC,962822,174282,40.645023,-74.077217,10301,56848.0
4,4,203063729,09/30/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1601001,F,Q,...,0,18-24,M,ASIAN / PACIFIC ISLANDER,1035521,188624,40.684266,-73.815138,11420,59832.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167960,167960,191717230,01/01/2019,259.0,"CRIMINAL MISCHIEF,UNCLASSIFIED 4",351.0,CRIMINAL MISCHIEF & RELATED OF,PL 1450001,M,Q,...,0,25-44,M,WHITE HISPANIC,1013749,204160,40.727008,-73.893572,11379,69843.0
167961,167961,191717236,01/01/2019,744.0,BAIL JUMPING 3,359.0,OFFENSES AGAINST PUBLIC ADMINI,PL 2155500,M,Q,...,0,25-44,M,WHITE HISPANIC,1032428,198872,40.712411,-73.826217,11415,63549.0
167962,167962,191717241,01/01/2019,339.0,"LARCENY,PETIT FROM OPEN AREAS,",341.0,PETIT LARCENY,PL 1552500,M,S,...,0,25-44,F,BLACK,952218,173159,40.641908,-74.115423,10310,61925.0
167963,167963,191719207,01/01/2019,109.0,"ASSAULT 2,1,UNCLASSIFIED",106.0,FELONY ASSAULT,PL 1200502,F,B,...,2,25-44,F,WHITE HISPANIC,1006918,238567,40.821466,-73.918101,10451,26754.0


# Crime genders

In [168]:
df['LAW_CAT_CD'].value_counts()

M    99997
F    64303
V     2345
I      358
Name: LAW_CAT_CD, dtype: int64

In [171]:
df['ARREST_BORO'].value_counts()

K    45842
M    41793
B    38309
Q    34864
S     7157
Name: ARREST_BORO, dtype: int64

# Arrest dates

In [170]:
df['ARREST_DATE'].value_counts()

05/02/2019    888
04/17/2019    874
01/10/2019    856
01/23/2019    838
07/24/2019    837
             ... 
01/21/2019    380
08/25/2019    377
09/02/2019    365
05/12/2019    362
09/29/2019    357
Name: ARREST_DATE, Length: 273, dtype: int64

# Crime types stats

In [165]:
df['OFNS_DESC'].describe()

count                           167848
unique                              64
top       ASSAULT 3 & RELATED OFFENSES
freq                             24687
Name: OFNS_DESC, dtype: object

In [83]:
df['OFNS_DESC'].value_counts()

ASSAULT 3 & RELATED OFFENSES       24687
PETIT LARCENY                      16800
DANGEROUS DRUGS                    16750
FELONY ASSAULT                     11610
VEHICLE AND TRAFFIC LAWS           11585
                                   ...  
DISRUPTION OF A RELIGIOUS SERV         4
HOMICIDE-NEGLIGENT-VEHICLE             3
PARKING OFFENSES                       3
NYS LAWS-UNCLASSIFIED VIOLATION        2
UNLAWFUL POSS. WEAP. ON SCHOOL         1
Name: OFNS_DESC, Length: 64, dtype: int64

In [142]:
robbery = df[df['OFNS_DESC']=='ROBBERY']
robbery

,index,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,...,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,zipcode,med_income
0,0,203078287,09/30/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1600500,F,M,...,0,25-44,M,BLACK HISPANIC,990563,203120,40.724200,-73.977226,10009,59929.0
3,3,203061218,09/30/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1601502,F,S,...,0,25-44,M,WHITE HISPANIC,962822,174282,40.645023,-74.077217,10301,56848.0
4,4,203063729,09/30/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1601001,F,Q,...,0,18-24,M,ASIAN / PACIFIC ISLANDER,1035521,188624,40.684266,-73.815138,11420,59832.0
27,27,203061114,09/30/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 160102A,F,K,...,0,18-24,M,BLACK,1001139,192775,40.695792,-73.939096,11206,28559.0
32,32,203061143,09/30/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1600500,F,M,...,0,18-24,M,BLACK,989869,207927,40.737395,-73.979725,10009,59929.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167827,167827,191735613,01/01/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1601504,F,K,...,0,25-44,M,BLACK,1018124,187785,40.682047,-73.877869,11208,35079.0
167850,167850,191717166,01/01/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 160102A,F,K,...,0,25-44,M,BLACK,999507,190206,40.688743,-73.944987,11216,43996.0
167874,167874,191733628,01/01/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1600500,F,Q,...,0,45-64,M,WHITE HISPANIC,1021183,214625,40.755704,-73.866693,11369,53617.0
167882,167882,191720920,01/01/2019,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1601504,F,K,...,0,25-44,M,BLACK,1018124,187785,40.682047,-73.877869,11208,35079.0


In [161]:
df1 = robbery[['OFNS_DESC','zipcode','med_income']]
df1.describe()

,med_income
count,6691.000000
mean,48024.697056
std,24097.694460
min,20232.000000
25%,32945.000000
50%,41328.000000
75%,56848.000000
max,170630.000000


In [160]:
df2 = df1[['OFNS_DESC','zipcode']]
df2

,OFNS_DESC,zipcode
0,ROBBERY,10009
3,ROBBERY,10301
4,ROBBERY,11420
27,ROBBERY,11206
32,ROBBERY,10009
...,...,...
167827,ROBBERY,11208
167850,ROBBERY,11216
167874,ROBBERY,11369
167882,ROBBERY,11208


In [158]:
df2.groupby("zipcode").count().sort_values('OFNS_DESC',ascending=False)

,OFNS_DESC
zipcode,
11208,284
10457,187
10454,160
10451,145
10459,133
...,...
11004,1
11105,1
11234,1


# Medium income info

In [163]:
df["med_income"].describe()

count    161057.000000
mean      49789.808459
std       25041.161536
min       20232.000000
25%       32945.000000
50%       43107.000000
75%       59229.000000
max      230952.000000
Name: med_income, dtype: float64